Notebook to generate a figure to illustrate feature importances

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#!/usr/bin/env python3

from itertools import product
from collections import deque

import os
import pandas as pd
import glob
import numpy as np

import datetime
from scipy import signal
import pywt

import pyarrow as pa
import pyarrow.parquet as pq
import json

import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import sys

import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, '../../bin/models/')
from nested_xval_utils import *

In [ ]:
fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')

plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = fsize-5
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

In [ ]:
fs={'feature':['psd_t'], 'stacking':['horizontal'], 'dims':[['H0','H1','UP']], 'augment':[True]}
feature_sets=[dict(zip(fs, v)) for v in product(*fs.values())]

#d = {'n_folds':[5],'max_depth': [50], 'n_estimators': [120], 'class_wt':[None],'wl_thresh':[0, 0.001,.005],}
d = {'n_folds':[5],'max_depth': [100], 'n_estimators': [120], 'class_wt':[None, "balanced_subsample"],'wl_thresh':[0]}
#d = {'n_folds':[5],'max_depth': [100], 'n_estimators': [500], 'class_wt':[None, "balanced_subsample"]}
hyperp=[dict(zip(d, v)) for v in product(*d.values())]

params=[i | feature_sets[0] for i in hyperp]

In [ ]:
params

In [ ]:
pq_list=[os.path.join('../../data/feature_sets/',f) for f in os.listdir(os.path.join( '../../data/feature_sets/'))]
#pd_list=[pd.read_parquet(pq) for pq in pq_list if ".pq" in pq]
meta_list=[read_meta(pq_fs) for pq_fs in pq_list if ".pq" in pq_fs]
meta_df=pd.DataFrame.from_records(meta_list)


######################
ambient_list= list(meta_df[meta_df.magnitude.isnull()].eq_name.unique())
event_list=meta_df[~meta_df.magnitude.isnull()].sort_values(['magnitude'], ascending=False).groupby("eq_name").count().sort_values(['station'], ascending=False).index.tolist()
full_list=ambient_list+event_list

#convert to rsn
full_list=meta_df[meta_df.eq_name.isin(full_list)].record_number.unique()

In [ ]:
X_train, y_train, name_list, times, snr_metric=list_to_featurearrays(full_list[:5], params[0], test=False)

In [ ]:
#add frequency information
freq_array=np.array([30.        , 15.        , 10.        ,  7.5       ,  6.        ,
        5.        ,  4.28571429,  3.75      ,  3.33333333,  3.        ,
        2.72727273,  2.5       ,  2.30769231,  2.14285714,  2.        ,
        1.875     ,  1.76470588,  1.66666667,  1.57894737,  1.5       ,
        1.42857143,  1.36363636,  1.30434783,  1.25      ,  1.2       ,
        1.15384615,  1.11111111,  1.07142857,  1.03448276])
strings = ["%.1fs" % number for number in list(freq_array)]
new_names=list(name_list[:6])+(strings)

In [ ]:
fig,ax=plt.subplots(2, 1,figsize=(8,6))

from matplotlib.patches import FancyBboxPatch
import matplotlib.transforms as mtransforms

#clf = joblib.load(os.path.join(os.path.dirname(os.getcwd()), 'data/model_all_40.pkl'))
clf = joblib.load('../../models/synth_model_all_43.pkl')
importances = list(clf.feature_importances_)
important_len=int(len(importances)/3)
val=35
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
#import matplotlib.pyplot as plt
#%matplotlib inline
# Set the style
#plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
my_colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
my_colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
direcs=['East','North','Up']
for i,color,direc in zip(np.arange(3),my_colors,direcs):
    ax[0].axvspan(i*(important_len), i*(important_len)+6, alpha=0.25, color=my_colors[3])
    ax[0].axvspan(i*(important_len)+6+29, i*(important_len)+6+29, alpha=0.25, color='grey')
    ax[0].bar(x_values[i*important_len:i*important_len+important_len], importances[i*important_len:i*important_len+important_len],  color=color, label=direc)
    
    
#ax[0].set_xlim([-1,248])
ax[0].legend()

#ax[0].fill_between(x, 0, 1, where=y > threshold,
#                color='green', alpha=0.5, transform=ax.get_xaxis_transform())



# Tick labels for x axis
ax[0].set_xticks([])
ax[0].set_ylabel('Importance')
trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)
ax[0].text(0, 1.0, '(a)', transform=ax[0].transAxes + trans,
            fontsize='medium', verticalalignment='top')

i=1
ax[1].axvspan(i*(important_len), i*(important_len)+6, alpha=0.25, color=my_colors[3])
ax[1].axvspan(i*(important_len)+6+29, i*(important_len)+6+29, alpha=0.25, color='grey')
ax[1].bar(x_values[i*important_len:i*important_len+val], importances[i*important_len:i*important_len+val], orientation = 'vertical', color=my_colors[i])
ax[1].set_ylabel('Importance')

ax[1].text(0, 1.0, '(b)', transform=ax[1].transAxes + trans,
            fontsize='medium', verticalalignment='top')
#ax[1].bar(x_values[80:105], importances[80:105], orientation = 'vertical')
# Tick labels for x axis
name_list2=['4th Max','3rd Max','2nd Max','** Max','Median','Mad', 'SNR','* 30s','* 15s','* 10s','* 7.5s','* 6s','* 5s','4.3s','3.8s','3.3s','3s','2.7s','2.5s', '2.3s']
#ax[1].set_xticks(x_values[i*82:i*82+18], name_list[i*82:i*82+30], rotation='vertical')
ax[1].set_xticks(x_values[i*important_len:(i+1)*important_len][::2], new_names[:important_len][::2], rotation='vertical')

#ax[1].text(x_values[i*important_len]+20,0.015, '<<increasing frequency<<')
t = ax[1].text(
    x_values[i*important_len]+20,0.03, "increasing frequency", ha="center", va="center", rotation=0, size=12,
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="white", ec="b", lw=1))
'''
t = ax[1].text(
    x_values[i*important_len]+50+80,0.015, "increasing scale", ha="center", va="center", rotation=0, size=12,
    bbox=dict(boxstyle="larrow,pad=0.3", fc="white", ec="b", lw=2))

'''

from matplotlib.transforms import (
    Bbox, TransformedBbox, blended_transform_factory)
from mpl_toolkits.axes_grid1.inset_locator import (
    BboxPatch, BboxConnector, BboxConnectorPatch)


def connect_bbox(bbox1, bbox2,
                 loc1a, loc2a, loc1b, loc2b,
                 prop_lines, prop_patches=None):
    if prop_patches is None:
        prop_patches = {
            **prop_lines,
            "alpha": prop_lines.get("alpha", 1) * 0.2,
        }

    c1 = BboxConnector(bbox1, bbox2, loc1=loc1a, loc2=loc2a, **prop_lines)
    c1.set_clip_on(False)
    c2 = BboxConnector(bbox1, bbox2, loc1=loc1b, loc2=loc2b, **prop_lines)
    c2.set_clip_on(False)

    bbox_patch1 = BboxPatch(bbox1, **prop_patches)
    bbox_patch2 = BboxPatch(bbox2, **prop_patches)

    p = BboxConnectorPatch(bbox1, bbox2,
                           # loc1a=3, loc2a=2, loc1b=4, loc2b=1,
                           loc1a=loc1a, loc2a=loc2a, loc1b=loc1b, loc2b=loc2b,
                           **prop_patches)
    p.set_clip_on(False)

    return c1, c2, bbox_patch1, bbox_patch2, p


def zoom_effect01(ax1, ax2, xmin, xmax, **kwargs):
    """
    Connect *ax1* and *ax2*. The *xmin*-to-*xmax* range in both axes will
    be marked.

    Parameters
    ----------
    ax1
        The main axes.
    ax2
        The zoomed axes.
    xmin, xmax
        The limits of the colored area in both plot axes.
    **kwargs
        Arguments passed to the patch constructor.
    """

    trans1 = blended_transform_factory(ax1.transData, ax1.transAxes)
    trans2 = blended_transform_factory(ax2.transData, ax2.transAxes)

    bbox = Bbox.from_extents(xmin, 0, xmax, 1)

    mybbox1 = TransformedBbox(bbox, trans1)
    mybbox2 = TransformedBbox(bbox, trans2)

    prop_patches = {**kwargs, "color":'#a6cee3',"ec": "none", "alpha": 0.1}

    c1, c2, bbox_patch1, bbox_patch2, p = connect_bbox(
        mybbox1, mybbox2,
        loc1a=3, loc2a=2, loc1b=4, loc2b=1,
        prop_lines=kwargs, prop_patches=prop_patches)

    ax1.add_patch(bbox_patch1)
    ax2.add_patch(bbox_patch2)
    ax2.add_patch(c1)
    ax2.add_patch(c2)
    ax2.add_patch(p)

    return c1, c2, bbox_patch1, bbox_patch2, p


zoom_effect01(ax[0], ax[1], 35, 35+6+29)

##################
fig.tight_layout()
#plt.savefig('figs/importances.png',dpi=300)
plt.show()